In [1]:
from datasets import load_dataset
import numpy as np 
import pandas as pd 
from sklearn.metrics import f1_score, accuracy_score
from datasets import load_dataset
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
fp16 = True if torch.cuda.is_available() else False

C:\Users\Anwender\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
label_columns = ['pornographic-content', 'violence', 'death', 'sexual-assault', 'abuse', 'blood',
                 'suicide', 'pregnancy', 'child-abuse', 'incest', 'underage', 'homophobia',
                 'self-harm', 'dying', 'kidnapping', 'mental-illness', 'dissection',
                 'eating-disorders', 'abduction', 'body-hatred', 'childbirth', 'racism',
                 'sexism', 'miscarriages', 'transphobia', 'abortion', 'fat-phobia',
                 'animal-death', 'ableism', 'classism', 'misogyny', 'animal-cruelty']

In [3]:
from transformers import AdamW, BigBirdTokenizer ,get_linear_schedule_with_warmup, BigBirdForSequenceClassification, Trainer, TrainingArguments,EvalPrediction, AutoTokenizer, BertForSequenceClassification, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                num_labels=32,
                problem_type="multi_label_classification",
                return_dict=True)

model.to(device)
model.load_state_dict(torch.load('saved_models_db_loss/model_epoch_2.pt'))
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [4]:
train = load_dataset("json", 
                     data_files="segments/train_preprocessed_segments.jsonl", 
                     #streaming=True, split="train")
                         )

val = load_dataset("json", 
                     data_files="segments/val_preprocessed_segments.jsonl", 
                     #streaming=True, split="train")
                         )

train

DatasetDict({
    train: Dataset({
        features: ['pornographic-content', 'violence', 'death', 'sexual-assault', 'abuse', 'blood', 'suicide', 'pregnancy', 'child-abuse', 'incest', 'underage', 'homophobia', 'self-harm', 'dying', 'kidnapping', 'mental-illness', 'dissection', 'eating-disorders', 'abduction', 'body-hatred', 'childbirth', 'racism', 'sexism', 'miscarriages', 'transphobia', 'abortion', 'fat-phobia', 'animal-death', 'ableism', 'classism', 'misogyny', 'animal-cruelty', 'segments', 'attention_masks'],
        num_rows: 307102
    })
})

In [5]:
from tqdm import tqdm
import jsonlines

train_output_file_path = "train_embeddings.jsonl"
val_output_file_path = "val_embeddings.jsonl"

def create_encodings(data, path):
    for document in tqdm(data["train"]):    
        segments = document['segments']
        attention_masks = document['attention_masks']
        labels = [label_tensor for label_tensor in {key: value for key, value in document.items() if key in label_columns}.values()]
        
        with torch.no_grad():
            tensor_segments = torch.tensor(segments).to(device)
            tensor_attention_masks = torch.tensor(attention_masks).to(device)
            
            outputs = model(tensor_segments, tensor_attention_masks, output_hidden_states=True)        
            cls_embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy().tolist() # extract class token
            
            with jsonlines.open(path, mode='a') as writer:
                output = {"embeddings": cls_embeddings, "labels": labels}
                writer.write(output) 

create_encodings(train, train_output_file_path)
create_encodings(val, val_output_file_path)

100%|████████████████████████████████████████████████████████████████████████████| 17104/17104 [15:18<00:00, 18.63it/s]


In [6]:
val_bert = load_dataset("json", 
                     data_files=val_output_file_path, 
                     streaming=True, split="train")
    
for data in val_bert:
    print(data)
    break 

{'embeddings': [[0.055512361228466034, 0.19034384191036224, -0.22573672235012054, -0.1817525327205658, 0.2890360653400421, 0.4910011887550354, 0.09496267139911652, 0.5983070731163025, -0.0763557180762291, -0.6667563915252686, 0.05256887152791023, -0.20466962456703186, -0.3115101158618927, -0.05270069092512131, -0.24165134131908417, 0.6548388600349426, 0.1893998384475708, -0.2678012549877167, -0.5453880429267883, 0.11992903053760529, -0.27138930559158325, -0.4800584316253662, -0.27060529589653015, 0.058536265045404434, 0.5021445155143738, -0.5793655514717102, -0.13243652880191803, -0.2650614380836487, 0.2698405086994171, 0.034649092704057693, 0.4360765218734741, -0.02582412213087082, 0.18991060554981232, -0.05734359472990036, 0.16741177439689636, -0.2618650197982788, -0.06632768362760544, -0.004338563419878483, -0.25652992725372314, 0.32752925157546997, 0.5097134113311768, -0.057353075593709946, 0.19749794900417328, 0.0637555792927742, -0.0884394720196724, -0.1087724044919014, -0.733948